# Functions to Manipulate and Transcribe Audio and Video Files

pattyry
Nov 14, 2020

### Trim a movie file

In [1]:
import os
import moviepy
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import subprocess
import scipy.io.wavfile as wavfile
import pandas as pd
import numpy as np
import os
from scipy.io import wavfile
import scipy.signal as sps

In [2]:
cwd = "c:\\"
os.chdir(cwd)
filein = "recording-in.mp4"
fileout = "recording-out.mp4"
os.getcwd()

'c:\\'

### Set start time and end time, and extract desired clip

In [98]:
start_time =30
end_time = 1000000

ffmpeg_extract_subclip(filein, start_time, end_time, targetname=fileout)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


### Extract wav file from video

In [104]:
command = "ffmpeg -i c:/iota/source/recording-out.mp4 -ab 160k -ac 2 -ar 22000 -vn recording-out.wav"

subprocess.call(command, shell=True)

1

### Chunk wav's to length limit of speech to text

In [221]:
os.chdir("c:\\transcription")
transcription = os.getcwd()
transcription

'c:\\iota\\transcription'

In [222]:
os.chdir("c:\\source")
source = os.getcwd()
source
os.listdir()

['recording-out.mp4', 'recording-out.wav']

In [223]:
from scipy.io.wavfile import read as read_wav

sampling_rate, data=read_wav("recording-out.wav") # enter your filename
sampling_rate

22000

In [ ]:
resample_rate = 16000

In [228]:
def _slice_audio_data(raw_audio_data, interval_len):
    accounted_fs = int(sampling_rate * interval_len)   
    num_slices = int(len(raw_audio_data) / accounted_fs)   
    audio_data = raw_audio_data[: int((num_slices) * accounted_fs)]  # Trims audio array    
    remnant = np.array(raw_audio_data[int((num_slices) * accounted_fs): ] )    
    sliced_audio = np.array(np.split(audio_data, (num_slices)))    
    row_length = np.array(np.zeros(sliced_audio[1].shape))     
    row_length[:remnant.shape[0],] = remnant  
    sliced_audio = np.vstack([sliced_audio, row_length])
    return np.array(sliced_audio)

In [231]:
for subdir, dirs, files in os.walk(source):
    for file in files:
        os.chdir(source)
        filepath = subdir + os.sep + file
        if filepath.endswith(".wav"):
            fs, data = wavfile.read(filepath)
            data = data.sum(axis=1) / 2 #convert from stereo to mono
            sliced_audio = _slice_audio_data(data, 600)
            sliced_audio = np.asarray(sliced_audio, dtype=np.int16)
            os.chdir(transcription)
            for i in range(sliced_audio.shape[0]):
                num = str(i)
                data = sliced_audio[i,:]
                # Resample data
                number_of_samples = round(len(data) * float(resample_rate) / sampling_rate)
                data = sps.resample(sliced_audio[i,:], number_of_samples)
                data = np.asarray(data, dtype=np.int16)
                # Write data
                wavfile.write(num+"-"+file, resample_rate, data)

### Speech to Text 

In [400]:
import time
import wave
import string
import azure.cognitiveservices.speech as speechsdk
import sys

In [401]:
sampling_rate, data=read_wav("0-recording-out.wav") # enter your filename
sampling_rate

16000

In [402]:
speech_key, service_region = "<my-speech-to-text-key", "<my-service-location"
speech_config=speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
recordings = os.listdir()
recordings

['0-recording-out.wav',
 '1-recording-out.wav',
 '2-recording-out.wav',
 '3-recording-out.wav',
 '4-recording-out.wav',
 '5-recording-out.wav',
 '6-recording-out.wav',
 '7-recording-out.wav',
 '8-recording-out.wav']

### Invoke Azure Cognitive Services Speech to Text Service Against a List of 10-minute Recordings.

In [443]:
def _speech_recognize_continuous_from_file(file, speech_key, service_region, count):
    
    all_results = []
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
    speech_config.speech_recognition_language="en-US"
    audio_config = speechsdk.audio.AudioConfig(filename=file)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    
    done = False

    def stop_cb(evt):
        """callback that stops continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        speech_recognizer.stop_continuous_recognition()
        nonlocal done
        done = True

    all_results = []
    def handle_final_result(evt):
        all_results.append(evt.result.text)

    speech_recognizer.recognized.connect(handle_final_result)
    
    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
    speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))

    speech_recognizer.session_stopped.connect(stop_cb)

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(1)

    with open(count+"all_results.txt", "w") as output:
        output.write(str(all_results))
        
    return all_results


In [ ]:
new_results = []
results = []
count = 0
for i in recordings:
    file = "c://iota//transcription//"+ i
    print(file)
    new_results = _speech_recognize_continuous_from_file(file, speech_key, service_region, count)
    count = count +1
    results.append(new_results)
    print(len(results))


In [ ]:
with open(count+"concatanated_results.txt", "w") as output:
    output.write(str(results))